# Visualizing Topics

The visualization will allow us to quickly see words that are most relevant to a topic and the distances between topics.

In [3]:
#!pip install --upgrade jupyter_client


In [1]:
#%pip install pyLDAvis

IMPORT LIBRARIES

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

import pyLDAvis
import pyLDAvis.lda_model

nltk.download("stopwords")
nltk.download("wordnet")

import warnings
warnings.filterwarnings("ignore", message=r"datetime\.datetime\.utcnow\(\) is deprecated")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


LOAD THE DATASET

In [4]:
document = pd.read_csv("/content/bbc-text.csv")
document.head()

,category,text
0,tech,tv future in the hands of viewers with home th...
1,business,worldcom boss left books alone former worldc...
2,sport,tigers wary of farrell gamble leicester say ...
3,sport,yeading face newcastle in fa cup premiership s...
4,entertainment,ocean s twelve raids box office ocean s twelve...


TEXT PROCESSING

In [5]:
stop_words = set(stopwords.words("english"))
lemmatizer = WordNetLemmatizer()

def process_text(text):
  text = text.lower()
  tokens = text.split()
  tokens = [word for word in tokens if word.isalpha()]
  tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]
  return " ".join(tokens)

document["clean_text"] = document["text"].astype(str).apply(process_text)

DOCUMENT-TERM MATRIX

In [7]:
vectorizer = CountVectorizer(
    max_df = 0.95,
    min_df = 5
)

dtm = vectorizer.fit_transform(document["clean_text"])

TRAIN LDA TOPIC MODEL

In [8]:
num_topics = 5

lda_model = LatentDirichletAllocation(
    n_components = num_topics,
    random_state = 42,
    learning_method = "batch"
)

lda_model.fit(dtm)

LatentDirichletAllocation(n_components=5, random_state=42)

DISPLAY TOPICS

In [10]:
feature_names = vectorizer.get_feature_names_out()

def display_topics(model, feature_names, n_words = 10):
  for idx, topic in enumerate(model.components_):
    print(f"\nTopic {idx + 1}:")
    print(" ".join([feature_names[i]
                    for i in topic.argsort()[:-n_words - 1:-1]]))

display_topics(lda_model, feature_names)


Topic 1:
game england player club would said wale back side first

Topic 2:
said year company firm market also bank would sale new

Topic 3:
film best year said one award also first last world

Topic 4:
said people game mobile technology phone new also one service

Topic 5:
mr said would government labour people party minister say blair


VISUALIZE TOPICS

In [12]:
pyLDAvis.enable_notebook()

lda_vis = pyLDAvis.lda_model.prepare(
    lda_model,
    dtm,
    vectorizer
)

lda_vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
4      0.070435 -0.078477       1        1  24.729402
3      0.085238  0.130326       2        1  22.616686
1      0.135456 -0.055335       3        1  20.593583
2     -0.131626  0.088615       4        1  18.940604
0     -0.159502 -0.085129       5        1  13.119725, topic_info=         Term         Freq        Total Category  logprob  loglift
4392       mr  2995.000000  2995.000000  Default  30.0000  30.0000
2564     film   988.000000   988.000000  Default  29.0000  29.0000
2787     game  1237.000000  1237.000000  Default  28.0000  28.0000
3773   labour   761.000000   761.000000  Default  27.0000  27.0000
2239  england   570.000000   570.000000  Default  26.0000  26.0000
...       ...          ...          ...      ...      ...      ...
7279     want   181.300934   706.285492   Topic5  -5.6495   0.6712
2852      get   184.320419   944.817418   Topic5  -5.6330   0.3967
1513    could   188.144817  1485.814576   Topic5  -5.6125  -0.0355
231      also   188.940760  2112.083045   Topic5  -5.6082  -0.3829
4487      new   186.547563  1921.203852   Topic5  -5.6210  -0.3010

[381 rows x 6 columns], token_table=      Topic      Freq        Term
term                             
51        3  0.977992  accounting
80        4  0.997988       actor
81        4  0.996059     actress
183       2  0.006676     airline
183       3  0.988075     airline
...     ...       ...         ...
7484      3  0.342469        year
7484      4  0.327354        year
7484      5  0.062353        year
7500      3  0.978900     yugansk
7502      3  0.993447       yukos

[766 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[5, 4, 2, 3, 1])